In [44]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt

img = cv.imread('Data Set\Answer Scripts\Answer Scripts\A\JJ502.jpg', cv.IMREAD_GRAYSCALE)
assert img is not None

scale_percent_w= 25  # Percentage of the original size width
scale_percent_h= 30  # Percentage of the original size height
width = int(img.shape[1] * scale_percent_w / 100)
height = int(img.shape[0] * scale_percent_h/ 100)
dim = (width, height)

# Resize image
resized_image = cv.resize(img, dim, interpolation=cv.INTER_AREA)

# size of the image
print(resized_image.shape)

imshow = cv.imshow('Image', resized_image)
cv.waitKey(0)
cv.destroyAllWindows()


(639, 772)


In [49]:
#extracting the  area of interest
x, y, w, h = 10, 180, 550, 350
roi = resized_image[y:y+h, x:x+w]

# Display the extracted region
cv.imshow('ROI', roi)
cv.waitKey(0)
cv.destroyAllWindows()

In [50]:
import cv2
import numpy as np

def extract_student_answers(image_path):
    #img = image_path
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize the image if it's too large (optional step)
    height, width = img.shape
    if height > 1000:
        img = cv2.resize(img, (width // 2, height // 2))
    
    # Apply Gaussian blur to smooth the image (helpful for better circle detection)
    blurred = cv2.GaussianBlur(img, (9, 9), 2.2)

    # Apply Hough Circle Transform to detect circles (bubbles)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=9, maxRadius=19)

    # If no circles are found, return an empty list
    if circles is None:
        print("No bubbles found!")
        return []

    # Convert circle coordinates to integers
    circles = np.round(circles[0, :]).astype("int")

    # Visualize the detected circles on the image for debugging
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for (x, y, r) in circles:
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)

    # Show the image with detected bubbles
    cv2.imshow("Detected Bubbles", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Sort the circles by their y-coordinate, and then x-coordinate (top-to-bottom, left-to-right)
    circles = sorted(circles, key=lambda c: (c[1], c[0]))

    # Define the number of questions and number of choices per question
    num_questions = 50
    num_choices = 5  # Assuming 5 choices per question (1-5)

    # Initialize a list to store detected answers
    student_answers = []

    # Group circles into batches of 5 (one batch per question)
    for q in range(num_questions):
        question_bubbles = circles[q * num_choices:(q + 1) * num_choices]
        
        # Sort the bubbles horizontally to map them to the correct answer options (1-5)
        question_bubbles = sorted(question_bubbles, key=lambda c: c[0])  # Sort left-to-right

        filled_option = None
        max_intensity = 0
        
        # Check which bubble is filled (darkest bubble)
        for idx, (x, y, r) in enumerate(question_bubbles):
            # Create a circular mask to isolate the bubble
            mask = np.zeros(img.shape, dtype="uint8")
            cv2.circle(mask, (x, y), r, 255, -1)

            # Use the mask to count the non-zero pixels inside the bubble
            bubble_intensity = cv2.mean(img, mask=mask)[0]

            # Assume the darkest bubble is the filled one (least intensity value)
            if bubble_intensity < max_intensity or filled_option is None:
                max_intensity = bubble_intensity
                filled_option = idx + 1  # Option is 1-indexed

        # Append the detected answer to the list
        student_answers.append(filled_option)

    return student_answers

# Example usage:
image_path = 'Data Set\Answer Scripts\Answer Scripts\A\JJ502.jpg'
#image_path = roi
student_answers = extract_student_answers(image_path)
print(student_answers)


[1, 1, 2, 1, 3, 1, 4, 1, 4, 4, 2, 3, 1, 1, 4, 4, 1, 1, 3, 2, 1, 4, 1, 5, 2, 3, 5, 1, 2, 3, 3, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
